We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [ ]:
import duckdb
from pathlib import Path

candidates = []
search_roots = [
    Path("."),
    Path(".."),
    Path("./data"),
    Path("../data"),
    Path("../../data"),
]

for root in search_roots:
    if root.exists():
        candidates.extend(list(root.glob("nflpa.duckdb")))
        candidates.extend(list(root.glob("**/nflpa.duckdb")))

seen = set()
duckdb_files = []
for f in candidates:
    fp = str(f.resolve())
    if fp not in seen:
        seen.add(fp)
        duckdb_files.append(f.resolve())

print("nflpa.duckdb candidates found", len(duckdb_files))
for i, f in enumerate(duckdb_files[:25]):
    print(i, f)

if not duckdb_files:
    raise RuntimeError("No nflpa.duckdb found near this notebook, run notebook 00 or check where you saved the database")

DB_PATH = duckdb_files[0]
print("using DB_PATH", DB_PATH)

con = duckdb.connect(str(DB_PATH))

con.execute("""
SELECT
  COUNT(*) AS n_tables
FROM (SHOW TABLES)
""").df()

Quick sanity check to confirm that the specialized database view has been successfully materialized and that it contains the complete set of features required for the analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

SEASON_COL = "season"
WEEK_COL = "week"
TEAM_COL = "team"

PANEL_TABLE = "team_week_panel"
MODEL_VIEW = "team_week_panel_nextweek_model"

con.execute("DROP TABLE IF EXISTS panel_next_week_flags")

con.execute(f"""
CREATE TABLE panel_next_week_flags AS
WITH base AS (
  SELECT
    {SEASON_COL} AS season,
    {WEEK_COL} AS week,
    CAST({TEAM_COL} AS VARCHAR) AS team_key
  FROM {PANEL_TABLE}
),
nxt AS (
  SELECT
    {SEASON_COL} AS season,
    {WEEK_COL} AS week,
    CAST({TEAM_COL} AS VARCHAR) AS team_key
  FROM {PANEL_TABLE}
)
SELECT
  b.season,
  b.week,
  b.team_key,
  CASE WHEN n.week IS NULL THEN 0 ELSE 1 END AS has_next_week
FROM base b
LEFT JOIN nxt n
  ON n.season = b.season
 AND n.team_key = b.team_key
 AND n.week = b.week + 1
""")

con.execute(f"DROP VIEW IF EXISTS {MODEL_VIEW}")

con.execute(f"""
CREATE VIEW {MODEL_VIEW} AS
SELECT
  p.*,
  f.has_next_week
FROM {PANEL_TABLE} p
JOIN panel_next_week_flags f
  ON f.season = p.{SEASON_COL}
 AND f.week = p.{WEEK_COL}
 AND f.team_key = CAST(p.{TEAM_COL} AS VARCHAR)
WHERE f.has_next_week = 1
""")

con.execute(f"""
SELECT
  COUNT(*) AS rows_model,
  COUNT(DISTINCT {SEASON_COL} || '-' || {WEEK_COL} || '-' || CAST({TEAM_COL} AS VARCHAR)) AS distinct_keys_model
FROM {MODEL_VIEW}
""").df()

We compute overall mean and variance for next week offensive and defensive injury outcomes and also adds basic distribution stats

In [ ]:
inj_summary = con.execute(f"""
SELECT
  COUNT(*) AS n_rows,
  AVG(CAST(Inj_Off_Next_w AS DOUBLE)) AS mean_inj_off_next,
  VAR_POP(CAST(Inj_Off_Next_w AS DOUBLE)) AS var_inj_off_next,
  MIN(CAST(Inj_Off_Next_w AS DOUBLE)) AS min_inj_off_next,
  approx_quantile(CAST(Inj_Off_Next_w AS DOUBLE), 0.25) AS p25_inj_off_next,
  approx_quantile(CAST(Inj_Off_Next_w AS DOUBLE), 0.50) AS p50_inj_off_next,
  approx_quantile(CAST(Inj_Off_Next_w AS DOUBLE), 0.75) AS p75_inj_off_next,
  MAX(CAST(Inj_Off_Next_w AS DOUBLE)) AS max_inj_off_next,

  AVG(CAST(Inj_Def_Next_w AS DOUBLE)) AS mean_inj_def_next,
  VAR_POP(CAST(Inj_Def_Next_w AS DOUBLE)) AS var_inj_def_next,
  MIN(CAST(Inj_Def_Next_w AS DOUBLE)) AS min_inj_def_next,
  approx_quantile(CAST(Inj_Def_Next_w AS DOUBLE), 0.25) AS p25_inj_def_next,
  approx_quantile(CAST(Inj_Def_Next_w AS DOUBLE), 0.50) AS p50_inj_def_next,
  approx_quantile(CAST(Inj_Def_Next_w AS DOUBLE), 0.75) AS p75_inj_def_next,
  MAX(CAST(Inj_Def_Next_w AS DOUBLE)) AS max_inj_def_next
FROM {MODEL_VIEW}
""").df()

inj_summary

We compute distribution summaries for special teams load measures for All, ScoreLinked, and NonScore, and create a long table to keep the stats consistent across buckets and makes comparisons easier

In [ ]:
con.execute("DROP TABLE IF EXISTS step12_st_load_summary")

con.execute(f"""
CREATE TABLE step12_st_load_summary AS
WITH long AS (
  SELECT '{'ST_Load_All_w'}' AS metric, CAST(ST_Load_All_w AS DOUBLE) AS x FROM {MODEL_VIEW}
  UNION ALL
  SELECT '{'ST_Load_ScoreLinked_w'}' AS metric, CAST(ST_Load_ScoreLinked_w AS DOUBLE) AS x FROM {MODEL_VIEW}
  UNION ALL
  SELECT '{'ST_Load_NonScore_w'}' AS metric, CAST(ST_Load_NonScore_w AS DOUBLE) AS x FROM {MODEL_VIEW}
)
SELECT
  metric,
  COUNT(*) AS n_rows,
  AVG(x) AS mean_x,
  VAR_POP(x) AS var_x,
  MIN(x) AS min_x,
  approx_quantile(x, 0.25) AS p25_x,
  approx_quantile(x, 0.50) AS p50_x,
  approx_quantile(x, 0.75) AS p75_x,
  MAX(x) AS max_x
FROM long
GROUP BY 1
ORDER BY 1
""")

con.execute("SELECT * FROM step12_st_load_summary ORDER BY metric").df()

We compute distribution summaries for special teams volatility measures for All, ScoreLinked, and NonScore, using the same table format as loads in order to help spot whether volatility is sparse or concentrated

In [ ]:
con.execute("DROP TABLE IF EXISTS step12_st_vol_summary")

con.execute(f"""
CREATE TABLE step12_st_vol_summary AS
WITH long AS (
  SELECT '{'ST_Vol_All_w'}' AS metric, CAST(ST_Vol_All_w AS DOUBLE) AS x FROM {MODEL_VIEW}
  UNION ALL
  SELECT '{'ST_Vol_ScoreLinked_w'}' AS metric, CAST(ST_Vol_ScoreLinked_w AS DOUBLE) AS x FROM {MODEL_VIEW}
  UNION ALL
  SELECT '{'ST_Vol_NonScore_w'}' AS metric, CAST(ST_Vol_NonScore_w AS DOUBLE) AS x FROM {MODEL_VIEW}
)
SELECT
  metric,
  COUNT(*) AS n_rows,
  AVG(x) AS mean_x,
  VAR_POP(x) AS var_x,
  MIN(x) AS min_x,
  approx_quantile(x, 0.25) AS p25_x,
  approx_quantile(x, 0.50) AS p50_x,
  approx_quantile(x, 0.75) AS p75_x,
  MAX(x) AS max_x
FROM long
GROUP BY 1
ORDER BY 1
""")

con.execute("SELECT * FROM step12_st_vol_summary ORDER BY metric").df()

We compute the proportion of weeks flagged as shock weeks for each shock definition and include the raw counts for interpretability

In [ ]:
shock_props = con.execute(f"""
SELECT
  COUNT(*) AS n_rows,
  SUM(CAST(ST_Shock_All_w AS BIGINT)) AS n_shock_all,
  AVG(CAST(ST_Shock_All_w AS DOUBLE)) AS p_shock_all,
  SUM(CAST(ST_Shock_ScoreLinked_w AS BIGINT)) AS n_shock_scorelinked,
  AVG(CAST(ST_Shock_ScoreLinked_w AS DOUBLE)) AS p_shock_scorelinked,
  SUM(CAST(ST_Shock_NonScore_w AS BIGINT)) AS n_shock_nonscore,
  AVG(CAST(ST_Shock_NonScore_w AS DOUBLE)) AS p_shock_nonscore
FROM {MODEL_VIEW}
""").df()

shock_props